<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=6887fc942ab19ad148a8312ec2bc4982271d20b53bc7218ddb19f8260a8b2725
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-07 09:15:28
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: 37.48 K (0.27%)
Current PnL: -20.17 L (-13.39%)
CY Booked + Current PnL: -8.73 L (-5.8%)
-------------------
Total profit:  1.51 L
Total loss:  -21.69 L
-------------------
Total Booked + Current PnL: 18.25 L (14.75%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.09%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.68 L (62.0%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 8.12%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115487.0,14983.0,14124.0,0.35,14.91,12.23,28.96,79.0,1.06,0.82,53.94,MH,ATH,METALS
77,TTKPRESTIG,770.00,92.74,38.0,M-SC,1.36,83028.0,-17749.0,17843.0,-0.25,-17.61,21.49,0.09,245.0,-0.99,0.59,9.08,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.93,189500.0,24830.0,18495.0,0.12,15.08,9.76,26.31,89.0,1.34,1.35,24.26,X40N,NTT,AC
19,CIPLA,1662.74,-21.84,46.0,H-LC,3.92,208350.0,3850.0,19439.0,0.51,1.88,9.33,11.39,10.0,0.20,1.48,11.00,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.04,49.0,H-LC,3.32,215280.0,4914.0,23875.0,0.36,2.34,11.09,13.68,37.0,0.21,1.53,19.25,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.34,54.0,H-LC,2.18,237330.0,-8821.0,88026.0,3.01,-3.58,37.09,32.18,16.0,-0.10,1.69,31.17,X200,ATH,IT
55,RAJOOENG,143.1,-38.56,49.0,H-SC,16.20,93160.0,-9340.0,49943.0,2.89,-9.11,53.61,39.61,114.0,-0.19,0.66,13.78,AR,ATH,MISC
11,BANKINDIA,190.0,-22.98,73.0,H-MC,10.91,197187.0,17379.0,95024.0,1.72,9.67,48.19,62.52,88.0,0.18,1.40,47.05,XR,NTT,BANKS
78,UJJIVANSFB,60.0,108.87,68.0,H-SC,13.38,132003.0,-10476.0,29991.0,1.14,-7.35,22.72,13.70,163.0,-0.35,0.94,55.80,OX40N,NTT,BANKS
63,SFL,1287.0,32.67,45.0,M-SC,6.24,175156.0,-87062.0,156887.0,1.03,-33.20,89.57,26.63,229.0,-0.55,1.24,8.90,XY24,NTT,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1241.31,67.0,M-SC,20.93,182502.0,1359.0,44111.0,-0.62,0.75,24.17,25.10,235.0,0.03,1.30,46.26,XY24,NTT,HEALTHCARE
84,VOLTAS,1530.0,-2.90,45.0,H-MC,1.40,203820.0,12078.0,25681.0,-0.58,6.30,12.60,19.69,99.0,0.47,1.45,13.52,XY25,NTT,AC
73,TATAMOTORS,1065.0,-18.21,57.0,M-LC,2.44,258692.0,-17831.0,127897.0,-0.48,-6.45,49.44,39.81,54.0,-0.14,1.84,23.98,XY24,NTT,AUTO
60,SAIL,228.0,44.35,51.0,M-MC,12.35,227321.0,2359.0,165285.0,-0.46,1.05,72.71,74.53,192.0,0.01,1.61,33.64,XY24,BTT,STEEL
56,RELAXO,1176.0,-39.49,38.0,H-SC,1.42,82513.0,-62647.0,138572.0,-0.36,-43.16,167.94,52.31,136.0,-0.45,0.59,10.45,X40N,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.0,1241.31,67.0,M-SC,20.93,182502.0,1359.0,44111.0,-0.62,0.75,24.17,25.10,235.0,0.03,1.30,46.26,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.0,-20.88,50.0,H-MC,2.05,136222.0,434.0,42324.0,0.38,0.32,31.07,31.48,107.0,0.01,0.97,12.48,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.79,46.0,M-LC,5.70,151187.0,242.0,109217.0,0.71,0.16,72.24,72.51,53.0,0.00,1.07,6.19,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,108.87,68.0,H-SC,13.38,132003.0,-10476.0,29991.0,1.14,-7.35,22.72,13.70,163.0,-0.35,0.94,55.80,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,58.0,H-SC,2.13,226152.0,-43515.0,79854.0,0.12,-16.14,35.31,13.47,138.0,-0.54,1.61,14.99,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,92.74,38.0,M-SC,1.36,83028.0,-17749.0,17843.0,-0.25,-17.61,21.49,0.09,245.0,-0.99,0.59,9.08,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,39.0,H-SC,1.51,141128.0,-34775.0,76802.0,0.21,-19.77,54.42,23.89,149.0,-0.45,1.00,21.88,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-23.88,45.0,H-MC,6.21,102885.0,-28950.0,71176.0,0.39,-21.96,69.18,32.03,98.0,-0.41,0.73,15.24,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1241.31,67.0,M-SC,20.93,182502.0,1359.0,44111.0,-0.62,0.75,24.17,25.10,235.0,0.03,1.30,46.26,XY24,NTT,HEALTHCARE
60,SAIL,228.00,44.35,51.0,M-MC,12.35,227321.0,2359.0,165285.0,-0.46,1.05,72.71,74.53,192.0,0.01,1.61,33.64,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,6.25,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.52,29.51,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.78,48.0,H-MC,2.93,189500.0,24830.0,18495.0,0.12,15.08,9.76,26.31,89.0,1.34,1.35,24.26,X40N,NTT,AC
84,VOLTAS,1530.00,-2.90,45.0,H-MC,1.40,203820.0,12078.0,25681.0,-0.58,6.30,12.60,19.69,99.0,0.47,1.45,13.52,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115487.0,14983.0,14124.0,0.35,14.91,12.23,28.96,79.0,1.06,0.82,53.94,MH,ATH,METALS
11,BANKINDIA,190.00,-22.98,73.0,H-MC,10.91,197187.0,17379.0,95024.0,1.72,9.67,48.19,62.52,88.0,0.18,1.40,47.05,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.25,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.52,29.51,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,37.0,H-SC,9.36,126421.0,3085.0,128545.0,0.39,2.50,101.68,106.72,119.0,0.02,0.90,26.46,AR,ATH,MISC
86,WIPRO,420.00,-13.79,46.0,M-LC,5.70,151187.0,242.0,109217.0,0.71,0.16,72.24,72.51,53.0,0.00,1.07,6.19,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,280082.0,-35760.0,148415.0,0.01,-11.32,52.99,35.67,5.0,-0.24,1.99,0.96,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.77,207583.0,-44185.0,97398.0,0.17,-17.55,46.92,21.13,27.0,-0.45,1.47,11.66,X40,ATH,PAINTS
74,TCS,4406.34,-29.65,44.0,H-LC,12.17,279913.0,-66043.0,134274.0,-0.35,-19.09,47.97,19.73,1.0,-0.49,1.99,3.10,X40,ATH,IT
42,ITC,452.00,-39.19,44.0,H-LC,2.13,196822.0,-3316.0,24209.0,0.44,-1.66,12.30,10.44,4.0,-0.14,1.40,3.81,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.37,46.0,H-LC,9.90,223191.0,-18733.0,80974.0,0.28,-7.74,36.28,25.73,8.0,-0.23,1.58,5.47,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.03,57.0,H-LC,2.00,162340.0,-23755.0,71235.0,-0.15,-12.76,43.88,25.51,15.0,-0.33,1.15,19.63,AR,ATH,ELECTRICAL
42,ITC,452.0,-39.19,44.0,H-LC,2.13,196822.0,-3316.0,24209.0,0.44,-1.66,12.30,10.44,4.0,-0.14,1.40,3.81,X40,NTT,FMCG
48,LTIM,7230.2,-4.34,54.0,H-LC,2.18,237330.0,-8821.0,88026.0,3.01,-3.58,37.09,32.18,16.0,-0.10,1.69,31.17,X200,ATH,IT
51,NESTLEIND,1377.0,-10.01,52.0,H-LC,3.06,276822.0,11396.0,45399.0,0.11,4.29,16.40,21.40,11.0,0.25,1.97,11.22,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.04,49.0,H-LC,3.32,215280.0,4914.0,23875.0,0.36,2.34,11.09,13.68,37.0,0.21,1.53,19.25,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-22.30,30.0,H-LC,9.96,280082.0,-35760.0,148415.0,0.01,-11.32,52.99,35.67,5.0,-0.24,1.99,0.96,X40N,ATH,FMCG
74,TCS,4406.34,-29.65,44.0,H-LC,12.17,279913.0,-66043.0,134274.0,-0.35,-19.09,47.97,19.73,1.0,-0.49,1.99,3.10,X40,ATH,IT
42,ITC,452.00,-39.19,44.0,H-LC,2.13,196822.0,-3316.0,24209.0,0.44,-1.66,12.30,10.44,4.0,-0.14,1.40,3.81,X40,NTT,FMCG
30,HCLTECH,1937.36,-2.37,46.0,H-LC,9.90,223191.0,-18733.0,80974.0,0.28,-7.74,36.28,25.73,8.0,-0.23,1.58,5.47,X40,ATH,IT
1,ABB,7934.00,-39.77,51.0,H-LC,7.51,251520.0,-10099.0,129306.0,0.42,-3.86,51.41,45.57,7.0,-0.08,1.79,7.02,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.03,57.0,H-LC,2.00,162340.0,-23755.0,71235.0,-0.15,-12.76,43.88,25.51,15.0,-0.33,1.15,19.63,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,44.0,H-LC,2.13,196822.0,-3316.0,24209.0,0.44,-1.66,12.30,10.44,4.0,-0.14,1.40,3.81,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-17.77,35.0,H-LC,5.77,207583.0,-44185.0,97398.0,0.17,-17.55,46.92,21.13,27.0,-0.45,1.47,11.66,X40,ATH,PAINTS
19,CIPLA,1662.74,-21.84,46.0,H-LC,3.92,208350.0,3850.0,19439.0,0.51,1.88,9.33,11.39,10.0,0.20,1.48,11.00,X40N,ATH,PHARMA
57,RELIANCE,1533.00,-14.04,49.0,H-LC,3.32,215280.0,4914.0,23875.0,0.36,2.34,11.09,13.68,37.0,0.21,1.53,19.25,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.65,44.0,H-LC,12.17,279913.0,-66043.0,134274.0,-0.35,-19.09,47.97,19.73,1.0,-0.49,1.99,3.10,X40,ATH,IT
40,INFY,2275.00,-18.88,50.0,H-LC,8.06,315346.0,2458.0,169215.0,0.30,0.79,53.66,54.87,3.0,0.01,2.24,7.49,X40,BTT,IT
42,ITC,452.00,-39.19,44.0,H-LC,2.13,196822.0,-3316.0,24209.0,0.44,-1.66,12.30,10.44,4.0,-0.14,1.40,3.81,X40,NTT,FMCG
83,VBL,671.64,-22.30,30.0,H-LC,9.96,280082.0,-35760.0,148415.0,0.01,-11.32,52.99,35.67,5.0,-0.24,1.99,0.96,X40N,ATH,FMCG
1,ABB,7934.00,-39.77,51.0,H-LC,7.51,251520.0,-10099.0,129306.0,0.42,-3.86,51.41,45.57,7.0,-0.08,1.79,7.02,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,44.0,L-SC,31.62,79162.0,-34387.0,74436.0,0.00,-30.28,94.03,35.27,268.0,-0.46,0.56,93.75,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,11.49,92400.0,-5580.0,27184.0,-0.05,-5.70,29.42,22.05,152.0,-0.21,0.66,34.31,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.74,115487.0,14983.0,14124.0,0.35,14.91,12.23,28.96,79.0,1.06,0.82,53.94,MH,ATH,METALS
78,UJJIVANSFB,60.00,108.87,68.0,H-SC,13.38,132003.0,-10476.0,29991.0,1.14,-7.35,22.72,13.70,163.0,-0.35,0.94,55.80,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,67.0,M-SC,12.48,144234.0,-4986.0,149167.0,0.20,-3.34,103.42,96.62,208.0,-0.03,1.02,56.09,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,65.0,H-SC,6.74,150756.0,-28315.0,94042.0,-0.27,-15.81,62.38,36.71,135.0,-0.30,1.07,49.58,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,67.0,M-SC,12.48,144234.0,-4986.0,149167.0,0.20,-3.34,103.42,96.62,208.0,-0.03,1.02,56.09,XY25,NTT,FINANCE
11,BANKINDIA,190.00,-22.98,73.0,H-MC,10.91,197187.0,17379.0,95024.0,1.72,9.67,48.19,62.52,88.0,0.18,1.40,47.05,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,6.25,214020.0,8944.0,103628.0,0.10,4.36,48.42,54.89,52.0,0.09,1.52,29.51,X5K,ATH,METALS
64,SHALBY,327.00,1241.31,67.0,M-SC,20.93,182502.0,1359.0,44111.0,-0.62,0.75,24.17,25.10,235.0,0.03,1.30,46.26,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.08
1,25,43.22
2,50,74.94


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.11
LC    32.19
MC    23.72
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.88
X40      14.57
X40N     11.22
XR       11.04
XY25     10.38
AR        8.23
OX40N     7.73
X200      1.69
MH        1.61
X5K       1.52
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.99
H-LC    25.64
H-MC    20.50
M-SC    13.70
M-LC     5.54
M-MC     2.92
L-SC     1.42
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.79,-5.75,40.60
IT,12.66,-17.94,81.82
FINANCE,10.42,-15.03,70.58
MISC,6.97,-16.84,81.58
BANKS,6.39,-10.82,69.12
PAINTS,5.57,-17.73,35.28
ELECTRICAL,5.36,-9.08,47.35
INSURANCE,3.75,-4.73,43.89
AUTO,3.54,-15.46,70.65


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3224704.0,22
XR,1355354.0,14
AR,1170468.0,9
X40,787100.0,10
X40N,657303.0,9
OX40N,557752.0,10
XY25,476967.0,7
SR,260473.0,2
X5K,103628.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3708934.0,29
M-SC,1500546.0,17
H-MC,1267645.0,15
H-LC,1186309.0,15
M-MC,371598.0,2
M-LC,369596.0,4
L-SC,261536.0,3
L-MC,60253.0,1
L-LC,41818.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1218737.0      6
M-SC       XY24       842288.0      7
H-SC       AR         801766.0      5
           XR         789177.0      7
H-MC       XY24       566651.0      4
H-LC       X40        506070.0      5
M-MC       XY24       371598.0      2
H-SC       X40N       307970.0      4
           SR         260473.0      2
           OX40N      258475.0      4
H-LC       X40N       216115.0      3
H-MC       X40        208348.0      4
H-LC       AR         200541.0      2
H-MC       XY25       179104.0      2
L-SC       XR         173023.0      2
M-SC       AR         168161.0      2
M-LC       XY24       156751.0      2
M-SC       XY25       149167.0      1
           OX40N      139588.0      4
H-MC       X40N       133218.0      2
M-SC       XR         128660.0      2
M-LC       XR         109217.0      1
H-LC       XY25       106878.0      3
M-LC       X5K        103628.0      1
H-MC       XR          95024.0      1
L-SC       OX40N       88513.0      1
H-LC       X200        88026.0      1
M-SC       X40         72682.0      1
H-SC       MH          72336.0      1
H-MC       OX40N       71176.0      1
H-LC       XY24        68679.0      1
L-MC       XR          60253.0      1
L-LC       XY25        41818.0      1
H-MC       MH          14124.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
